# acud Python session demo

There are mainly two ways to run an acud simulation:
- through the command line, or
- from within a Python interpreter session/Python script by importing acud as a Python package

This notebook emulates using acud interactively through the Python interpreter to query an acud solution. With minimal edits,
the code contained here could also be used as a Python script.

Text included in the notebook refers to the cell(s) immediately below. Using the pandas function *head*,
only the first rows are shown for time series that are too long to display. 

In [ ]:
import os
from math import ceil, floor

import numpy as np
import pandas as pd
from numpy import array as ndarray
from pandas import Series, DataFrame, MultiIndex, Index
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
!pip install -U matplotlib  # use if matplotlib import above fails

In [ ]:
import acud

In [ ]:
acud.__version__

List all functions and attributes exposed by the acud Python interface

In [ ]:
dir(acud)

### Case 102

Toy model

In [ ]:
os.chdir('102')

#### On-drive data sourcing for post processing

Raw solution results are read from files stored on disk and loaded in memory into an *xarray Dataset* object; further transformations are then applied using this *Dataset* object as output data source.

The acud *read_solution* also includes reporting capability, so when using this function there is no need to perform low-level post-processing.

In [ ]:
sol = acud.read_solution('output')
sol

Solution query with a mix of xarray and pandas syntax

In [ ]:
sol.prices.to_dataframe().unstack(level=0).xs('Bus1', level='bus_id', axis=1).head()

Two ways of using *sel* xarray method to run the same query 

In [ ]:
DataFrame(sol.prices.sel(bus_id='Bus1').to_dataframe().prices).head()

In [ ]:
sol.prices.sel(bus_id='Bus1', drop=('bus_id')).to_dataframe().head(24).plot();

In [ ]:
sol.use.sel(bus_id='Bus1').values

In [ ]:
sol.use.sel(bus_id='Bus1').plot();


In [ ]:
sol.dispatch.sum(dim='gen_id').plot();

In [ ]:
sol.dispatch.sel(gen_id='Gen1').plot();

In [ ]:
sol.dispatch.to_dataframe().unstack(level=0).plot(legend=None);

In [ ]:
sol.dispatch.to_dataframe().unstack(level=0).head(24).plot.bar(legend=None, stacked=True);

In [ ]:
df = sol.dispatch.to_dataframe().dispatch.unstack(level=0)
df.head(24)

In [ ]:
sol.flow.to_dataframe().head()

In [ ]:
sol.prices

In [ ]:
prices = sol.prices.sel(bus_id='Bus1').to_dataframe().prices.to_frame()
prices.plot();

In [ ]:
prices.head()

Generating units net revenue

In [ ]:
netrevenue = sol.netrevenues.sum(dim='period').to_dataframe().sort_values(by=['netrevenues'], ascending=True)
netrevenue.plot(kind='barh', color='green');

Generating units generation costs

In [ ]:
(sol.gencosts.sum(dim='period').to_dataframe()
 .sort_values(by=['gencosts'], ascending=False)
 .iloc[0:10].plot(kind='bar'));

Aggregated generation

In [ ]:
sol.dispatch.sum(dim='gen_id').to_dataframe().plot();

Bids by period and generating unit

In [ ]:
bids = sol.bids.to_dataframe()
bids.unstack(level=0).iloc[:,0:9].head()

Capacity factor by period and generating unit

In [ ]:
capfactor = sol.generation / sol.icapacity
capfactor.name = 'capacity factor'

In [ ]:
capfactor

Interactive plotting

In [ ]:
# imports for interactive plotting

import plotly
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

print plotly.__version__

Generation stack for a given day

In [ ]:
genstack = sol.generation.sel(period='2012-01-01').to_dataframe()
genstack = (genstack.loc[genstack.generation > 0]
            .unstack(level=0)
            .generation)

data = [go.Bar(
    x=genstack.index,
    y=genstack[gen],
    name=gen
)  for gen in genstack.columns]

layout = go.Layout(
    barmode='stack',
    xaxis=dict(
        showticklabels=False
    )
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)